Experimenting with Bayesian optimisation to produce adversarial examples.


References:
- General guide to bayesian optimization:
    https://distill.pub/2020/bayesian-optimization/
- Paper on using bayesian optimization in hard label settings:
https://arxiv.org/pdf/2007.07210.pdf
- Another paper with a different objective function that could be used:
https://arxiv.org/abs/1807.04457

Things to consider:
- Bayesian optimisation performs best on problems with a low number of dimensions (eg under 20) - our problem has on the scale of ~100 dimensions, which is still significantly less than image recognition problems. Other techniques (dimensional upsampling) can be used to use bayesian optimization on high dimension problems - may or may not be necessary here
- Search space needs to be defined - initially take mins/max of training data for each field? This will later be refined (eg changing derived fields along with postcode)
- Initial samples from which adversarial examples are to be generated - just take failed companies from the test data?
- skopt includes callbacks - monitor process of the optimisation/terminate early when we find an adversarial example.

Scikit-optimize: library for optimization, including functions for Bayesian optimization.

In [ ]:
import skopt
from skopt.plots import plot_convergence
import os
import pickle
import pandas as pd
import numpy as np

Step 1: defining search space.
Initially do so by taking min/max for each field in test data.

In [ ]:
#Load in test data
data_dir = os.getcwd()
with open(os.path.join(data_dir, "Processing", r"train_preproc.p"), 'rb') as data_file:
    train_data = pickle.load(data_file)
X_train, y_train = train_data
# forgot in preprocessing: convert from bool to int
X_train['hasGNotice'] = X_train['hasGNotice'].apply(int)

In [ ]:
min_max = pd.DataFrame(index=["min", "max"], columns=X_train.columns)

for column in X_train.columns:
    min_max[column]["max"] = X_train[column].max(axis=0)
    min_max[column]["min"] = X_train[column].min(axis=0)


pd.set_option("display.max_rows", 345)

In [ ]:
#load in model to attack
model = pickle.load(open(os.path.join("fitted_models", "xg_0.927"), "rb"))

In [ ]:
# Need to get search space from min_max array
search_space = list(map(tuple, min_max.transpose().to_records(index=False)))

In [ ]:
# weight some features differently in distance function
weighting_vector = np.array([1/(col.mean()+0.01) for col in [X_train[col_name] for col_name in X_train.columns]])

In [ ]:
def build_df(X):
    return pd.DataFrame(data=[X], columns=X_train.columns)

def get_arr(X):
    return X.values[0]

In [ ]:
# TODO: standardise all features so that this is just absolute distance
def distance_function(X, company, weighting_vector):
    d = abs(X-company)
    return (d @ weighting_vector)//1000

In [ ]:
""" ideas:
1) constrain search space to [ex - delta, ex + delta] either manually or by specifying the search space
Have the objective function just return the failed probability, and optimise it until it is small
2) add reasonable distance function to objective


Model wrapper:
 - find postcode fields from postcode
 - round boolean fields

"""
def compute_objective(X, true_company):
    X_df = build_df(X)
    delta = distance_function(X, get_arr(true_company), weighting_vector)
    failed_prob = model.predict_proba(X_df)[0,1]
    print("delta: ", delta)
    obj = failed_prob * (delta + 0.5)
    print("objective: ", obj)
    if model.predict(X_df)[0] == 0:
        print("model successfully fooled into thinking company is safe")
        print("probability of failing: %s" % failed_prob)
        return -1

    print("model not tricked")
    print("probability of failing: %s" % failed_prob)
    print()
    return obj

In [ ]:
class FinishedCallBack(skopt.callbacks.EarlyStopper):
    def __call__(self, result):
      """
      Messy code
      Parameters
      ----------
      result : `OptimizeResult`, scipy object
          The optimization as a OptimizeResult object.
      """
      return self._criterion(result)
    def _criterion(self, result):
        print(result.fun)
        if(result.fun == -1):
            print("I should stop now")
        return (result.fun == -1)

In [ ]:
failed_companies = X_train.loc[y_train == 1]

 ##### Actual Bayesian optimization:

In [ ]:
starting_example = failed_companies.iloc[:1]
x0 = list(get_arr(starting_example))
def objective(X):
    return compute_objective(X, true_company=starting_example)

# sometimes gets error here - keep rerunning and it will work after a while.
cb = FinishedCallBack()
res = skopt.gp_minimize(objective, search_space, x0 = x0, y0 = objective(x0), callback=cb)
plot_convergence(res)
minimum = res.x


In [1]:
# Check the ditance to the original example
distance_function(minimum, get_arr(starting_example), weighting_vector)




NameError: name 'distance_function' is not defined